### scBasset on Mouse Brain dataset

In [1]:
# change working directory to parent
import os
os.chdir("../")

# for debugging
from IPython.core.debugger import set_trace

# for loading functions and classes from model and training related files
import sys 
sys.path.insert(1,"./scBasset/")
sys.path.insert(1,"./scBasset/scBasset")

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/mcb/users/dcakma3/bin/miniconda3/envs/moETM/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from scBasset.dataloader import load_mouse_brain_dataset, prepare_mouse_brain_dataset
from scBasset.scBasset.train import train_scBasset

Please set the following variables to train the model

In [4]:
mod1_file_path = f"./data/mouse_brain/rna_count.preprocessed.h5ad"
mod2_file_path = f"./data/mouse_brain/atac_count.preprocessed.h5ad"
genome_dir = "./data/mm10"

# random state for weight initialization and dataset shuffling
seed = 35

# training on GPU/CPU
# if you want to train the model on GPU (recommended) 
# set `gpu` to True and set gpu_id
gpu = True
gpu_id = 2

# saving related 
dataset = "mouse_brain"
result_dir = f"./experiments/scBasset/{dataset}/results"
model_dir = f"./experiments/scBasset/{dataset}/models"

# appended to the path on result_dir
eval_figures_suffix = "eval_figures"

#### Load Mouse brain data

In [8]:
# load mouse brain data
adata_mod1, adata_mod2 = load_mouse_brain_dataset(mod1_file_path, mod2_file_path,
    genome_dir=genome_dir, verbose=True) 
adata_mod1, adata_mod2 = prepare_mouse_brain_dataset(adata_mod1, adata_mod2)

> Loading genome...
> Extracting sequences underlying peaks...


#### Train model

In [7]:
if torch.cuda.is_available() and gpu:
    device = torch.device(f"cuda:{gpu_id}")
    print(f"> GPU {gpu_id} is available...")
else:
    device = torch.device("cpu")
    print("> GPU is not available... Switching to CPU")

> GPU 2 is available...


In [8]:
# set model hyperparameters
model_hps = {}
model_hps["seq_embed_dim"] = 128
model_hps["seq_shift_max"] = 3
model_hps["seq_offset"] = 30
model_hps["first_conv_out_filters"] = 288
model_hps["first_conv_kernel_size"] = 17
model_hps["first_conv_pool_size"] = 3
model_hps["tower_conv_out_filters"] = 512
model_hps["tower_conv_kernel_size"] = 5
model_hps["tower_conv_pool_size"] = 2
model_hps["tower_conv_repeat"] = 6 
model_hps["channel_conv_out_filters"] = 256
model_hps["weight_init"] = "kaiming_normal"

In [9]:
# set training hyperparameters
train_hps = {}
train_hps["eta"] = 3e-4
train_hps["bs"] = 512
train_hps["opt"] = "adam"
train_hps["max_epoch"] = 1000
train_hps["es_p"] = 5
train_hps["wd"] = 0.001
train_hps["device"] = device
train_hps["seed"] = seed
train_hps["summary"] = True
train_hps["model_dir"] = model_dir
train_hps["result_dir"] = result_dir

In [10]:
# set evaluation parameters
eval_params = {}
eval_params['batch_col'] = 'batch_indices'
eval_params['plot_fname'] = 'moETM'
eval_params['cell_type_col'] = 'cell_type'
eval_params['clustering_method'] = 'louvain'
eval_params['resolutions'] = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
eval_params['plot_dir'] = f'{train_hps["result_dir"]}/eval_figs'

In [11]:
# train the model
train_scBasset(model_hps, train_hps, adata_mod2, eval_params)

Layer (type:depth-idx)                        Output Shape              Param #
scBasset                                      --                        --
├─StochasticReverseComplement: 1-1            [512, 1404, 4]            --
├─StochasticShift: 1-2                        [512, 1344, 4]            --
├─SequenceEncoder: 1-3                        [512, 128]                --
│    └─Conv2dBlock: 2-1                       [512, 288, 448]           --
│    │    └─Conv2d: 3-1                       [512, 288, 1344, 1]       19,872
│    │    └─BatchNorm1d: 3-2                  [512, 288, 1344]          576
│    │    └─MaxPool1d: 3-3                    [512, 288, 448]           --
│    └─Conv1dTower: 2-2                       [512, 512, 7]             --
│    │    └─Sequential: 3-4                   [512, 512, 7]             4,299,411
│    └─Conv1dBlock: 2-3                       [512, 256, 7]             --
│    │    └─GELU: 3-5                         [512, 512, 7]             --
│    │  